# (PISA 2012 Dataset)
## by (Gad Mohamed)




---




### **Table of contents**
* Data Wrangling
* Uni-variate exploration 
* Bi-variate exploration 
* Multi-variate exploration

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

In [ ]:
#don't print output
%%capture 

#download PISA2012 dataset
!wget 'https://s3.amazonaws.com/udacity-hosted-downloads/ud507/pisa2012.csv.zip'
!unzip pisa2012.csv.zip


#download PISA2012 dictionary
!wget 'https://s3.amazonaws.com/udacity-hosted-downloads/ud507/pisadict2012.csv'


## Data wrangling

### read and inspect the dataset

In [2]:
pisa_df = pd.read_csv('pisa2012.csv', encoding = 'cp1252')
pisa_dict_df = pd.read_csv('pisadict2012.csv', encoding = 'cp1252')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,16,17,21,22,23,24,25,26,30,31,36,37,45,65,123,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,475) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pisa_df.head()

,Unnamed: 0,CNT,SUBNATIO,STRATUM,OECD,NC,SCHOOLID,STIDSTD,ST01Q01,ST02Q01,ST03Q01,ST03Q02,ST04Q01,ST05Q01,ST06Q01,ST07Q01,ST07Q02,ST07Q03,ST08Q01,ST09Q01,ST115Q01,ST11Q01,ST11Q02,ST11Q03,ST11Q04,ST11Q05,ST11Q06,ST13Q01,ST14Q01,ST14Q02,ST14Q03,ST14Q04,ST15Q01,ST17Q01,ST18Q01,ST18Q02,ST18Q03,ST18Q04,ST19Q01,ST20Q01,...,W_FSTR45,W_FSTR46,W_FSTR47,W_FSTR48,W_FSTR49,W_FSTR50,W_FSTR51,W_FSTR52,W_FSTR53,W_FSTR54,W_FSTR55,W_FSTR56,W_FSTR57,W_FSTR58,W_FSTR59,W_FSTR60,W_FSTR61,W_FSTR62,W_FSTR63,W_FSTR64,W_FSTR65,W_FSTR66,W_FSTR67,W_FSTR68,W_FSTR69,W_FSTR70,W_FSTR71,W_FSTR72,W_FSTR73,W_FSTR74,W_FSTR75,W_FSTR76,W_FSTR77,W_FSTR78,W_FSTR79,W_FSTR80,WVARSTRR,VAR_UNIT,SENWGT_STU,VER_STU
0,1,Albania,80000,ALB0006,Non-OECD,Albania,1,1,10,1.0,2,1996,Female,No,6.0,"No, never","No, never","No, never",None,None,1.0,Yes,Yes,Yes,Yes,NaN,NaN,<ISCED level 3A>,No,No,No,No,"Other (e.g. home duties, retired)",<ISCED level 3A>,NaN,NaN,NaN,NaN,Working part-time <for pay>,Country of test,...,4.3313,4.5084,4.5084,13.7954,13.9235,4.3389,13.0829,13.9235,4.3389,4.3313,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,4.5084,4.5315,13.0829,4.5315,4.3313,4.5084,4.5084,13.7954,13.9235,4.3389,13.0829,13.9235,4.3389,4.3313,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,19,1,0.2098,22NOV13
1,2,Albania,80000,ALB0006,Non-OECD,Albania,1,2,10,1.0,2,1996,Female,"Yes, for more than one year",7.0,"No, never","No, never","No, never",One or two times,None,1.0,Yes,Yes,NaN,Yes,NaN,NaN,<ISCED level 3A>,Yes,Yes,No,No,Working full-time <for pay>,<ISCED level 3A>,No,No,No,No,Working full-time <for pay>,Country of test,...,4.3313,4.5084,4.5084,13.7954,13.9235,4.3389,13.0829,13.9235,4.3389,4.3313,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,4.5084,4.5315,13.0829,4.5315,4.3313,4.5084,4.5084,13.7954,13.9235,4.3389,13.0829,13.9235,4.3389,4.3313,13.7954,13.9235,13.1249,13.1249,4.3389,13.0829,19,1,0.2098,22NOV13
2,3,Albania,80000,ALB0006,Non-OECD,Albania,1,3,9,1.0,9,1996,Female,"Yes, for more than one year",6.0,"No, never","No, never","No, never",None,None,1.0,Yes,Yes,No,Yes,No,No,"<ISCED level 3B, 3C>",Yes,Yes,Yes,No,Working full-time <for pay>,<ISCED level 3A>,Yes,No,Yes,Yes,Working full-time <for pay>,Country of test,...,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,4.2436,4.2436,12.7307,4.2436,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13
3,4,Albania,80000,ALB0006,Non-OECD,Albania,1,4,9,1.0,8,1996,Female,"Yes, for more than one year",6.0,"No, never","No, never","No, never",None,None,1.0,Yes,Yes,No,Yes,No,No,"<ISCED level 3B, 3C>",No,No,No,No,Working full-time <for pay>,<ISCED level 3A>,Yes,Yes,No,No,Working full-time <for pay>,Country of test,...,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,4.2436,4.2436,12.7307,4.2436,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13
4,5,Albania,80000,ALB0006,Non-OECD,Albania,1,5,9,1.0,10,1996,Female,"Yes, for more than one year",6.0,"No, never","No, never","No, never",One or two times,None,2.0,Yes,Yes,Yes,NaN,NaN,NaN,She did not complete <ISCED level 1>,No,No,No,No,Working part-time <for pay>,"<ISCED level 3B, 3C>",No,No,No,Yes,Working part-time <for pay>,Country of test,...,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,4.2436,4.2436,12.7307,4.2436,4.2436,4.2436,4.2436,12.7307,12.7307,4.2436,12.7307,12.7307,4.2436,4.2436,12.7307,12.7307,12.7307,12.7307,4.2436,12.7307,19,1,0.1999,22NOV13


In [4]:
pisa_dict_df.head()

,Unnamed: 0,x
0,CNT,Country code 3-character
1,SUBNATIO,Adjudicated sub-region code 7-digit code (3-di...
2,STRATUM,Stratum ID 7-character (cnt + region ID + orig...
3,OECD,OECD country
4,NC,National Centre 6-digit Code


### reducing the massive dataset to include only the relevant columns to our problem.

In [6]:
pisa_dict_df.rename(columns = {'Unnamed: 0': 'code'}, inplace = True)

In [52]:
#columns that I will work about in this project
codes = ['CNT', 'STIDSTD', 'ST13Q01', 'ST17Q01', 'ST57Q01', 'ST57Q02', 'ST57Q03', 'ST57Q04', 'ST57Q05', 'MMINS', 'LMINS', 'SMINS', 'PV1MATH', 'PV2MATH', 'PV3MATH',
           'PV4MATH', 'PV5MATH', 'PV1READ', 'PV2READ',  'PV3READ', 'PV4READ', 'PV5READ', 'PV1SCIE', 'PV2SCIE', 'PV3SCIE', 'PV4SCIE', 'PV5SCIE']

#the names of these codes in the PISA dictionary csv
names = pisa_dict_df.loc[pisa_dict_df.code.isin(codes)].x.values

In [53]:
#create a much smaller dataset that contains only relevant columns and renamed them to their original names
clean_pisa = pisa_df[codes].rename(columns = {code:name for code, name in zip(codes, names)})
clean_pisa.head()

,Country code 3-character,Student ID,Mother<Highest Schooling>,Father<Highest Schooling>,Out-of-School Study Time - Homework,Out-of-School Study Time - Guided Homework,Out-of-School Study Time - Personal Tutor,Out-of-School Study Time - Commercial Company,Out-of-School Study Time - With Parent,Learning time (minutes per week) - <test language>,Learning time (minutes per week)- <Mathematics>,Learning time (minutes per week) - <Science>,Plausible value 1 in mathematics,Plausible value 2 in mathematics,Plausible value 3 in mathematics,Plausible value 4 in mathematics,Plausible value 5 in mathematics,Plausible value 1 in reading,Plausible value 2 in reading,Plausible value 3 in reading,Plausible value 4 in reading,Plausible value 5 in reading,Plausible value 1 in science,Plausible value 2 in science,Plausible value 3 in science,Plausible value 4 in science,Plausible value 5 in science
0,Albania,1,<ISCED level 3A>,<ISCED level 3A>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406.8469,376.4683,344.5319,321.1637,381.9209,249.5762,254.3420,406.8496,175.7053,218.5981,341.7009,408.8400,348.2283,367.8105,392.9877
1,Albania,2,<ISCED level 3A>,<ISCED level 3A>,NaN,NaN,6.0,0.0,0.0,270.0,315.0,90.0,486.1427,464.3325,453.4273,472.9008,476.0165,406.2936,349.8975,400.7334,369.7553,396.7618,548.9929,471.5964,471.5964,443.6218,454.8116
2,Albania,3,"<ISCED level 3B, 3C>",<ISCED level 3A>,NaN,6.0,6.0,7.0,2.0,NaN,300.0,NaN,533.2684,481.0796,489.6479,490.4269,533.2684,401.2100,404.3872,387.7067,431.3938,401.2100,499.6643,428.7952,492.2044,512.7191,499.6643
3,Albania,4,"<ISCED level 3B, 3C>",<ISCED level 3A>,10.0,2.0,2.0,0.0,0.0,135.0,135.0,90.0,412.2215,498.6836,415.3373,466.7472,454.2842,547.3630,481.4353,461.5776,425.0393,471.9036,438.6796,481.5740,448.9370,474.1141,426.5573
4,Albania,5,She did not complete <ISCED level 1>,"<ISCED level 3B, 3C>",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381.9209,328.1742,403.7311,418.5309,395.1628,311.7707,141.7883,293.5015,272.8495,260.1405,361.5628,275.7740,372.7527,403.5248,422.1746


In [54]:
clean_pisa.shape

(485490, 27)

In [51]:
clean_pisa['Student ID']


,Out-of-School Study Time - Homework,Out-of-School Study Time - Guided Homework,Out-of-School Study Time - Personal Tutor,Out-of-School Study Time - Commercial Company,Out-of-School Study Time - With Parent,Learning time (minutes per week) - <test language>,Learning time (minutes per week)- <Mathematics>,Learning time (minutes per week) - <Science>,Plausible value 1 in mathematics,Plausible value 2 in mathematics,Plausible value 3 in mathematics,Plausible value 4 in mathematics,Plausible value 5 in mathematics,Plausible value 1 in reading,Plausible value 2 in reading,Plausible value 3 in reading,Plausible value 4 in reading,Plausible value 5 in reading,Plausible value 1 in science,Plausible value 2 in science,Plausible value 3 in science,Plausible value 4 in science,Plausible value 5 in science
Student ID,,,,,,,,,,,,,,,,,,,,,,,
1,6.128205,1.771429,0.702703,0.702703,1.394737,220.837838,231.513514,191.030303,475.345257,476.719851,470.167609,473.237537,472.802247,474.820601,481.050301,477.649997,474.907528,481.091521,470.791463,475.700731,471.765088,479.512965,475.865296
2,4.476190,1.193548,1.390244,0.921053,1.650000,253.634146,225.357143,203.314286,469.281000,466.886904,468.513513,464.412631,467.299284,460.990626,457.307063,463.421169,461.619499,462.209269,461.986304,462.905082,463.316469,464.632924,462.644532
3,6.071429,1.800000,1.095238,1.214286,1.093023,207.357143,206.000000,191.820513,475.143644,468.900693,472.955747,475.613306,477.457544,474.855065,470.509841,472.886135,475.172026,482.166818,475.555387,469.850746,475.006856,472.223104,477.091240
4,6.631579,2.028571,0.750000,1.371429,1.135135,213.789474,224.100000,199.742857,487.587174,491.848419,486.372940,488.102649,489.591794,488.809637,479.706990,481.486897,481.154544,485.584451,489.190268,492.042584,487.215593,483.595349,485.638590
5,5.162791,1.225000,1.000000,0.853659,1.195122,214.594595,223.717949,187.727273,486.015556,480.494259,484.835685,486.760125,486.748668,487.002334,485.242763,491.743362,486.999422,489.137259,480.695032,480.914446,488.141225,489.498812,487.894387


In [35]:
clean_pisa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 27 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   Country code 3-character                             485490 non-null  object 
 1   Mother<Highest Schooling>                            485490 non-null  int64  
 2   Father<Highest Schooling>                            457979 non-null  object 
 3   Out-of-School Study Time - Homework                  443261 non-null  object 
 4   Out-of-School Study Time - Guided Homework           301367 non-null  float64
 5   Out-of-School Study Time - Personal Tutor            269808 non-null  float64
 6   Out-of-School Study Time - Commercial Company        283813 non-null  float64
 7   Out-of-School Study Time - With Parent               279657 non-null  float64
 8   Learning time (minutes per week)  - <test language>  2

### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!